In [54]:
## Importing the required Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from geopy.geocoders import Nominatim

In [55]:
address = 'Vancouver, Canada'
geolocator = Nominatim(user_agent="anduleh")
location = geolocator.geocode(address)
vancity_latitude = location.latitude
vancity_longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address,latitude, longitude))

The geograpical coordinate of Vancouver, Canada are 49.2331424, -123.1177695.


In [57]:
df = pd.read_csv('ftp://webftp.vancouver.ca/OpenData/csv/cov_localareas.csv')

In [62]:
clean_df = df.rename({'NAME':'Neighbourhood'}, axis='columns').drop('MAPID',axis=1)
clean_df['Latitude'] = ''
clean_df['Longitude'] = ''
for i, row in clean_df.iterrows():
    address = '{}, Vancouver, Canada'.format(row['Neighbourhood'])
    location = geolocator.geocode(address)
    clean_df.set_value(i,'Latitude',location.latitude)
    clean_df.set_value(i,'Longitude',location.longitude)

C:\Users\Andrew\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
C:\Users\Andrew\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [63]:
clean_df

,Neighbourhood,Latitude,Longitude
0,Sunset,49.2196,-123.09
1,Mount Pleasant,49.2633,-123.097
2,Riley Park,49.2474,-123.103
3,Downtown,49.2834,-123.117
4,Kitsilano,49.2694,-123.155
5,Dunbar-Southlands,49.2535,-123.185
6,Kerrisdale,49.2347,-123.155
7,Arbutus-Ridge,49.241,-123.167
8,West Point Grey,49.2645,-123.185
9,Marpole,49.2092,-123.136


In [64]:
map_vancouver = folium.Map(location=[vancity_latitude, vancity_longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(clean_df['Latitude'], clean_df['Longitude'], clean_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_vancouver)  
    
map_vancouver